# Mul  클래스 구현

In [77]:

class Variable:
    def __init__(self, data, name =None):
        
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} 은(는) 지원하지 않습니다.')        
                
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0
    
    @property
    def ndim(self):
        return self.data.ndim
    
    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype
    
    @property
    def shape(self):
        return self.data.shape
    
    def __len__(self):
        return len(self.data)
    def __repr__(self):
        if self.data is None:
            return 'vairable(None)'
        
        p = str(self.data).replace('\n', '\n'+ ' '*9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
    
    def backward(self, retain_grad = False):
    
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if  f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x : x.generation)
                
        add_func(self.creator)        
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                
                if x.grad is None: 
                    x.grad = gx
                    
                else:
                    x.grad = x.grad + gx
                    
                if x.creator is not None:
                    add_func(x.creator)
                    
                    
        
            if not  retain_grad:
                for y in f.outputs:
                    y().grad = None #y 는 약한 참조(weakref)

In [78]:

class Config:
    enable_backprop = True
    
import weakref

class Function(object):
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        # 추가 부분 ===================================================================================
        if Config.enable_backprop: # config  데이터는 한 군데에만 존재하는게 좋기때문에 인스턴스화 하지 않고, 클래스 상태로 이용합니다.
        # ===============================================================================================
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)

            self.inputs = inputs
            ## 수정 부분 ========================================================================================================
            self.outputs = [weakref.ref(output) for output in outputs]
            ## ==================================================================================================================
        return  outputs if len(outputs) > 1 else outputs[0]

In [89]:
class Square(Function):
    def forward(self, x):
        y = x**2
        return y

    def backward(self, gy):
        
        x = self.inputs[0].data
        gx = 2 * x*gy
        return gx
    
    

def square(x):
    return Square()(x)    
    
class Add(Function):
    def forward(self, x0, x1):
        print(x0, x1)
        y = x0 + x1
        return y
    def backward(self, gy):
        return gy, gy
    
    
def as_array(x):
    if np.isscalar(x):
        return np.array((x))
    return x
def add(x0, x1):
    return Add()(x0, x1)

In [96]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y
    
    def backward(self, gy):
        x0, x1  = self.inputs[0].data, self.inputs[1].data
        return gy * x1 , gy * x0

In [97]:
def mul(x0, x1):
    return Mul()(x0, x1)

In [82]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a,b), c)

y.backward()

print(y)
print(a.grad)
print(b.grad)

6.0 1.0
variable(7.0)
2.0
3.0


## 매번 위에 처럼 add(mul(a,b),c) 보다 a*b +c 가 더 편할것 같습니다.

In [83]:

class Variable:
    def __init__(self, data, name =None):
        
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} 은(는) 지원하지 않습니다.')        
                
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0
    
    @property
    def ndim(self):
        return self.data.ndim
    
    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype
    
    @property
    def shape(self):
        return self.data.shape
    
    def __len__(self):
        return len(self.data)
    def __repr__(self):
        if self.data is None:
            return 'vairable(None)'
        
        p = str(self.data).replace('\n', '\n'+ ' '*9)
        return 'variable(' + p + ')'
    
    def __mul__(self, other):
        return mul(self, other)
    
    def __add__(self, other):
        return add(self, other)
    

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
    
    def backward(self, retain_grad = False):
    
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if  f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x : x.generation)
                
        add_func(self.creator)        
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                
                if x.grad is None: 
                    x.grad = gx
                    
                else:
                    x.grad = x.grad + gx
                    
                if x.creator is not None:
                    add_func(x.creator)
                    
                    
        
            if not  retain_grad:
                for y in f.outputs:
                    y().grad = None #y 는 약한 참조(weakref)

In [84]:
### 연산자 *를 오버로드 하였습니다. 곱셈의 특수 메서드는 __mul__(self, other) 입니다.

In [85]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
y  = a * b
print(y)

variable(6.0)


#
- 위의 예에서 a*b 가 실행되면 먼저 인스턴스 a 의 특수 메서드인  __mul__이  호출 됩니다. 그런데 만약 a에 __mul__ 메서드가 구현되어 있지 않으면 인스턴스 b의 * 연산자 특수 메서드가 호출 됩니다. 이 경우 b는 * 연산자의 오른쪽에 위치하기 때문에 __mul__이 아닌 __rmul__이라는 특수 메서드가 호출됩니다.

## + 연산자도 함께 오버로드 하겠습니다.

In [86]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)

6.0 1.0
variable(7.0)
2.0
3.0


In [ ]:
# obj 가 Variable 인스턴스 또는 ndarray 인스턴스라고 가정합니다. obj가 Variable 인스턴스면 아무것도 손보지 않고 그대로 반환하고, 
# 그렇지 않으면 Variable 인스턴스로 변환하여 반환합니다.
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    
    return Variable(obj)

In [87]:
class Function(object):
    def __call__(self, *inputs):
        # 수정 부부 ==============================
        inputs = [as_variable(x) for  x in inputs]
        xs = [x.data for x in inputs]
        # =======================================
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        if Config.enable_backprop: # config  데이터는 한 군데에만 존재하는게 좋기때문에 인스턴스화 하지 않고, 클래스 상태로 이용합니다.
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)

            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]
        return  outputs if len(outputs) > 1 else outputs[0]

In [90]:
x  =  Variable(np.array(2.0))
y =  x+ np.array(3.0)
print(y)

2.0 3.0
variable(5.0)


# float int 와 함께 사용하기

In [91]:
def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

In [92]:
x = Variable(np.array(2.0))
y = x + 3.0
print(y)

2.0 3.0
variable(5.0)


### 문제점 1 : 첫 번째 인수가 float나 int 인 경우

In [93]:
y = 2.0 * x


TypeError: unsupported operand type(s) for *: 'float' and 'Variable'

In [101]:
def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)

In [102]:
Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul


In [104]:
x = Variable(np.array(2.0))
y = 3.0* x + 1.0
print(y)

6.0 1.0
variable(7.0)


### 문제점 2. 좌항이 ndarray 인스턴스인 경우

In [105]:
x = Variable(np.array([1.0]))
y = np.array([2.0])+x

[1.] 2.0


In [107]:
y

array([variable([3.])], dtype=object)

# 
- 위의 예에서 좌항은 ndarray 인스턴스이고 우항은 Variable 인스턴스입니다. 이렇게 되면 좌항인 ndarray 인스턴스의 __add__ 메서드가 호출 됩니다. 하지만 우리는 우항인 Variable 인스턴스의 _radd__ 메서드가 호출되길 원합니다. 그러려면 '연산자 우선순위'를 지정해야 합니다.
__array_priority__ 를 추가하고 그값 을 큰 정수로 설정해야 합니다.

In [108]:
class Variable:
    __array_priority__ =200